In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

/Users/jesperpaulsen/.conda/envs/tdt4265/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jesperpaulsen/.conda/envs/tdt4265/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jesperpaulsen/.conda/envs/tdt4265/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [3]:
#config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"
#config_file = "configs/mnist_tdt4265_server_experimental.yaml"
# config_file = "configs/train_rdd2020_server.yaml"
config_file = "configs/train_rdd2020_server_experimental.yaml"
cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-19 14:58:50,404 SSD INFO: Loaded configuration file configs/train_rdd2020_server_experimental.yaml
2021-04-19 14:58:50,406 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    THRESHOLD: 0.5
    BACKBONE:
        NAME: 'res_net'
        PRETRAINED: True
        OUT_CHANNELS: [256, 512, 256, 256, 128, 128]
        INPUT_CHANNELS: 3
    PRIORS:
        MIN_SIZES: [[15, 15], [45, 45], [111, 111], [162, 162], [213, 213], [264, 264]]
        MAX_SIZES: [[45, 45], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]
        FEATURE_MAPS: [[63, 38], [32, 19], [16, 10], [8, 5], [4, 3], [2, 1]]
        STRIDES: [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]]

INPUT:
    IMAGE_SIZE: [500, 300]
DATASETS:
    TRAIN: ("rdd2020_train",)
    TEST: ("rdd2020_val", )
SOLVER:
    MAX_ITER: 120000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 1e-3
OUTPUT_DIR: 'outputs/rdd2020_experimentals'
DATASET_DIR: "datasets"
2021-04-19 14:58:50,407 SSD INFO: Running with config:
DATASETS:
  TEST: ('rd

In [5]:
!pwd 
model = start_train(cfg)

/Users/jesperpaulsen/school/TDT4265-Project/assignment4/SSD
Detector initialized. Total Number of params:  18.26M
Backbone number of parameters: 17.59M
SSD Head number of parameters: 663.8K
2021-04-19 15:00:04,277 SSD.trainer INFO: No checkpoint found.


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/RDD2020_filtered/ImageSets/train.txt'

In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-15 10:20:59,235 SSD INFO: Start evaluating...
2021-04-15 10:21:01,146 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):


100%|██████████| 100/100 [00:04<00:00, 24.52it/s]


2021-04-15 10:21:06,107 SSD.inference INFO: mAP: 0.8952
0               : 0.9038
1               : 0.8619
2               : 0.8991
3               : 0.9034
4               : 0.9012
5               : 0.9012
6               : 0.9021
7               : 0.8900
8               : 0.9006
9               : 0.8886



[{'metrics': {'mAP': 0.8952024131918288,
   '0': 0.9038319683421214,
   '1': 0.8619123396332643,
   '2': 0.8991184457164805,
   '3': 0.9034108127494757,
   '4': 0.9012078430736787,
   '5': 0.9011639146270728,
   '6': 0.9021376905405668,
   '7': 0.8900135604682246,
   '8': 0.9006312166832495,
   '9': 0.8885963400841522}}]